# version update colors

In [ ]:
# import depencies
from scipy.sparse import csr_matrix
import tables
import h5py
import hdf5plugin

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import seaborn as sb
from scipy import signal
import glob
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d as gfilter
import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

# scale figure
mpl.rcParams['figure.dpi'] = 200

# Import contact frequency matrices

In [ ]:
# import find raw data files

file_names = []
for name in glob.glob('./Raw data/*_5kb.h5'):
    file_names.append(name)

In [ ]:
file_names

# Define functions useful for analysis


In [ ]:
# import HiC heat map data as a matrix

#argument:
# filename: input name of the file you wish to import matrix for

#output:
#the heat map itself as a matrix
def make_matrix(filename):
    with tables.open_file(filename, 'r') as f:
        parts = {}
        try:
            for matrix_part in ('data', 'indices', 'indptr', 'shape'):
                parts[matrix_part] = getattr(f.root.matrix, matrix_part).read()
        except Exception as e:
            log.info('No h5 file. Please check parameters concerning the file type!')
            e
        matrix = csr_matrix(tuple([parts['data'], parts['indices'], parts['indptr']]),
                            shape=parts['shape'])
        matrix_array = matrix.toarray()
    return matrix_array

In [ ]:
#import interval information, contains list of indices and chromosome that they correspond to

#argument:
# filename: input name of the file you wish to import intervals for

#output:
#interval_list: list of chromosomes, indices
#keychr: keys to access different list from interval_list

def get_intervals(filename):
    file = h5py.File(filename)
    key_list = file['intervals'].keys()
    interval_list = {}
    for key in key_list:
        interval_list[key] = file['intervals'][key][()]
    keychr = list(key_list)
    return interval_list, keychr

In [ ]:
#generate a matrix for specific chromosome vs chromosome interaction

#argument:
#ma: matrix of heatmap
#chromosome_x: chromosome to use for x axis
#chromosome_y: chromosome to use for y axis

#output:
#matrix of desired region

def call_region(ma,chromosome_x,chromosome_y):
    start_x = indices[chromosome_x]['start']
    end_x = indices[chromosome_x]['end']
    
    start_y = indices[chromosome_y]['start']
    end_y = indices[chromosome_y]['end']
    
    return  ma[start_x:end_x,start_y:end_y]

In [ ]:
# chunky code to get the end index for regions, requried for get_indices function
# argument:
# chrlist: full list of which chromosome each index correspond to

#output:
#end: end index for each chromosome

def get_end_idx(chrlist):
    start = {}
    end = {}
    i = 0
    for chromes in chrlist:
        if chromes == b'chrIV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXVI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXIV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrX':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrIX':
            end[chromes] = i
            i+=1
        elif chromes == b'chrIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrM':
            end[chromes] = i
            i+=1
        else:
            break
    return end

In [ ]:
# makes a dictionary of each chromosomes start and end index

# argument:
# chromosomes: the list of chromosomes
# end_idx: output of get_end_idx

#output:
#idxs: dictionary of each chromosomes start and end index

def get_indices(chromosomes,end_idx):
    idxs = {}
    next_start = 0
    for chromes in chromosomes:
        idxs[chromes] = {}
        idxs[chromes]['end'] = end_idx[chromes]+1

        if chromes == str(b'chrIV'):
            idxs[chromes]['start'] = 0
        else:
            idxs[chromes]['start'] = next_start
        next_start = end_idx[chromes]+1
        
    return idxs

In [ ]:
#divide 2 matrices

#argument: 
#nom_matrix: matrix on the nominator
#denom_matrix: matrix on the denominator
#log2: whether to apply log2 to output matrix

#output:
#matrix: matrix after division

def divide_matrices(nom_matrix, denom_matrix, log2 = True):
    divide_matrix = nom_matrix/denom_matrix
#     divide_matrix[~np.isfinite(divide_matrix)] = 0
    if log2:
        matrix = np.log2(divide_matrix)
        matrix[~np.isfinite(matrix)] = 0
    else:
        matrix = divide_matrix
        matrix[~np.isfinite(matrix)] = 0
    return matrix

In [ ]:
#make the triangular matrix a full matrix by duplicating its diagonal counterpart

#argument:
#ma: matrix of heatmap

#output:
# full_matrix

def make_symmetric(ma):
    rotmatrix = np.transpose(ma)
    np.fill_diagonal(rotmatrix,0)
    full_matrix = ma + rotmatrix
    return full_matrix

In [ ]:
# import h5 files and produce the full matrix

#argument:
#filename: file name of the data to import

#output:
# full_matrix

def make_symm_matrix(filename):
    with tables.open_file(filename, 'r') as f:
        parts = {}
        try:
            for matrix_part in ('data', 'indices', 'indptr', 'shape'):
                parts[matrix_part] = getattr(f.root.matrix, matrix_part).read()
        except Exception as e:
            log.info('No h5 file. Please check parameters concerning the file type!')
            e
        matrix = csr_matrix(tuple([parts['data'], parts['indices'], parts['indptr']]),
                            shape=parts['shape'])
        matrix_array = matrix.toarray()
        
        full_ma = make_symmetric(matrix_array)

    return full_ma

In [ ]:
# 2d filter the matrix, eliminate high frequency noise

#argument:
#ma: matrix
#filter_length: length to apply the filter, larger length decreases max signal frequency

#output:
#filtered_ma: matrix after apply filtering

def filtering(ma, filter_length = 5):
    filtered_ma = signal.spline_filter(ma,lmbda=filter_length)
    return filtered_ma

In [ ]:
# plot the matrix as a heatmap

def plot_matrix(ma, filt = None,filter_len=0, symmetric = True, hlim = None,vlim =None, center=False, interpolation =None, cmap = 'RdYlBu_r', fig_size = 15,vcenter=None, vmin = None, vmax = None ):
    fig, ax = plt.subplots()

    ma[~np.isfinite(ma)] = 0
    if filt:
        filtered_ma = filtering(ma,filter_length = filter_len)
    else:
        filtered_ma = ma
    if symmetric:
        full_ma = filtered_ma
    else:
        full_ma = make_symmetric(filtered_ma)
    if center:
        norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
        im = ax.imshow(full_ma,cmap = cmap,norm=norm)
    else:
        im = ax.imshow(full_ma,cmap = cmap,interpolation = interpolation,vmin = vmin, vmax = vmax)
    plt.ylim(vlim)
    plt.xlim(hlim)
    cb = fig.colorbar(im,fraction=0.046, pad=0.04)
#     cb.set_ticks([0,1,2,3,4])
#     cb.set_ticklabels(['$10^0$','$10^1$','$10^2$','$10^3$','$10^4$'])
#     plt.xlabel('sites /5kB')
#     plt.ylabel('sites /5kB')

In [ ]:
# use this to average only the nonzero values. To include also empty values, use np.average(ma)
# note this can also be done for 1d array
#argument:
#ma: matrix of interest

#output:
# averaged value of matrix
def finite_average(ma):
    return np.sum(ma)/np.count_nonzero(ma)

# Import data


In [ ]:
#get the interval and key information, choose which file to use with file_names

interval ,keychr = get_intervals(file_names[0])
print(interval)
print(keychr)

In [ ]:
# list of chromosomes
chromosomes = [b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI',b'chrM']

In [ ]:
# get start and end indices corresponding to each chromosomes

indices = get_indices(chromosomes,get_end_idx(interval[keychr[0]]))

print(indices)

In [ ]:
# import list of genes from rna-seq analysis
ds2 = pd.read_csv('ds2h_idx.txt', delimiter='\t')

# indices of of genes from rna-seq analysis
chr_idx = ds2.groupby('chromosome_name')['idx'].apply(list).to_dict()

#list of chromosomes
chr_list = [b'chrI', b'chrII', b'chrIII', b'chrIV', b'chrV', b'chrVI', b'chrVII', b'chrVIII', b'chrIX', b'chrX', b'chrXI', b'chrXII', b'chrXIII', b'chrXIV', b'chrXV', b'chrXVI',b'chrM']

chr_idx2 = dict(zip(chr_list, list(chr_idx.values())))

In [ ]:
# convinience definations to label and index chromosomes
chrome_plots = ['IV','XV','VII','XII','XVI','XIII','II','XIV','X','XI','V','VIII','IX','III','VI','I']
chromosomes = [b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI']

chromosomes_num= [4, 15, 7, 12, 16, 13, 2, 14, 10, 11, 5, 8, 9, 3, 6, 1]

chromosomes_interval = []
for i in indices.keys():
    if i != b'chrM':
        inter = round((indices[i]['start'] + indices[i]['start'])/2)
        chromosomes_interval.append(inter)
end = indices[b'chrM']['end']

chromosome_label = []
for i in chromosomes_num:
    chromosome_label.append(str(i))
# chromosome_label.append('M')

In [ ]:
file_names

In [ ]:
AA20 = make_symm_matrix(file_names[0])
AA22 = make_symm_matrix(file_names[1])
AA25 = make_symm_matrix(file_names[2])
AA30 = make_symm_matrix(file_names[3])
AA32 = make_symm_matrix(file_names[4])
AA35 = make_symm_matrix(file_names[5])
AA40 = make_symm_matrix(file_names[6])
AA42 = make_symm_matrix(file_names[7])
AA45 = make_symm_matrix(file_names[8])

# AA20 = make_symm_matrix(file_names[5])
# AA22 = make_symm_matrix(file_names[3])
# AA25 = make_symm_matrix(file_names[6])
# AA30 = make_symm_matrix(file_names[4])
# AA32 = make_symm_matrix(file_names[2])
# AA35 = make_symm_matrix(file_names[7])
# AA40 = make_symm_matrix(file_names[0])
# AA42 = make_symm_matrix(file_names[1])
# AA45 = make_symm_matrix(file_names[8])

# Ratio of different heatmaps

In [ ]:
# log2 ratio of different replicates and time points
AA22_AA20 = divide_matrices(AA22,AA20,True)
AA25_AA20 = divide_matrices(AA25,AA20,True)

AA32_AA30 = divide_matrices(AA32,AA30,True)
AA35_AA30 = divide_matrices(AA35,AA30,True)

AA42_AA40 =  divide_matrices(AA42,AA40,True)
AA45_AA40 = divide_matrices(AA45,AA40,True)

AA40_AA30 =  divide_matrices(AA40,AA30,True)
AA40_AA20 = divide_matrices(AA40,AA20,True)

# ratio of different replicates and time points
AA22_AA20_nl = divide_matrices(AA22,AA20,False)
AA25_AA20_nl = divide_matrices(AA25,AA20,False)

AA32_AA30_nl = divide_matrices(AA32,AA30,False)
AA35_AA30_nl = divide_matrices(AA35,AA30,False)

AA42_AA40_nl =  divide_matrices(AA42,AA40,False)
AA45_AA40_nl = divide_matrices(AA45,AA40,False)

In [ ]:
from matplotlib.ticker import LogFormatter 
from matplotlib.ticker import LogFormatterExponent # <-- one new import here


------

## Averaging matrices

In [ ]:
# average heatmaps for 0 hour time point

AAave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([ AA20[r, c], AA30[r, c], AA40[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AAave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AAave[~np.isfinite(AAave)] = 0

In [ ]:
# average heatmaps for 2 hour time point

AA2ave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([AA22[r, c], AA32[r, c], AA42[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AA2ave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AA2ave[~np.isfinite(AA2ave)] = 0

In [ ]:
# average heatmaps for 5 hour time point

AA5ave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([AA25[r, c], AA35[r,c], AA45[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AA5ave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AA5ave[~np.isfinite(AA5ave)] = 0

In [ ]:
# ratio of 2 hour/DMSO and 5hour/DMSO

AAave2_0 = divide_matrices(AA2ave,AAave,log2=True)
AAave5_0 = divide_matrices(AA5ave,AAave,log2=True)

# Telomere

In [ ]:
#plot settings

mpl.rc('xtick', labelsize=12) 
mpl.rc('ytick', labelsize=12)
mpl.rc('figure',figsize = [10.0, 10.0])
# matplotlib.rc('axes',titlesize='large')

font = {'size'   : 12}

mpl.rc('font', **font)

from matplotlib.ticker import LogFormatter 
from matplotlib.ticker import LogFormatterExponent # <-- one new import here

In [ ]:
# obtain an average interaction around a telomere

def get_tel(matrix,volcano=3):
    volcano_map = np.zeros((2*len(chromosomes),2*len(chromosomes)))

    for c in enumerate(chromosomes):
        for d in enumerate(chromosomes):
            reg = call_region(matrix,c[1],d[1])
            reg_a = finite_average(reg[0:volcano,0:volcano])
            reg_b = finite_average(reg[0:volcano,-volcano:])
    #         reg_b = 0

            reg_c = finite_average(reg[-volcano:,0:volcano])
    #         reg_c = 0

            reg_d = finite_average(reg[-volcano:,-volcano:])
            volcano_map[2*c[0],2*d[0]] = reg_a
            volcano_map[2*c[0],2*d[0]+1] = reg_b
            volcano_map[2*c[0]+1,2*d[0]] = reg_c
            volcano_map[2*c[0]+1,2*d[0]+1] = reg_d
    return volcano_map

In [ ]:
# heatmap of telomere 2hr/DMSO ratio

AAave20_tel = get_tel(AAave2_0,volcano=3)

testlist=[i.decode('utf-8', 'ignore') for i in chromosomes]

chrom_romlist = [testlist[i][3:] for i in range(0,len(testlist))]

_=plt.imshow((AAave20_tel),cmap = 'bwr',vmax=2,vmin=-2)
_= plt.colorbar(fraction=0.046, pad=0.04)

#plt.title('Telomere 2hr/DMSO', fontsize = 20)
_=plt.xticks(np.arange(0,32,2),labels = chrom_romlist, rotation = 90)
_=plt.yticks(np.arange(0,32,2),labels = chrom_romlist)


mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

# plt.savefig('hic234plots/telomere_map_2hr', dpi=200)

In [ ]:
# heatmap of telomere 5hr/DMSO ratio

AAave50_tel = get_tel(AAave5_0,volcano=3)

_=plt.imshow((AAave50_tel),cmap = 'bwr',vmax=2,vmin=-2)
_= plt.colorbar(fraction=0.046, pad=0.04)

#plt.title('Telomere 5hr/DMSO')

_=plt.xticks(np.arange(0,32,2),labels = chrom_romlist, rotation = 90)
_=plt.yticks(np.arange(0,32,2),labels = chrom_romlist)

mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)
# plt.savefig('hic234plots/telomere_map_5hr', dpi=200)

-----

# Centromere

In [ ]:
# get interaction around centromere
def get_cent(matrix, volcano=3):
    volcano_map = np.zeros((len(chromosomes),len(chromosomes)))
    for c in enumerate(chromosomes):
        for d in enumerate(chromosomes):
            reg = call_region(matrix,c[1],d[1])
            cent_index_a = centromere[c[0]]
            cent_index_b = centromere[d[0]]
            reg_mean = finite_average(reg[cent_index_a-volcano:cent_index_a+volcano+1,cent_index_b-volcano:cent_index_b+volcano+1])
            volcano_map[c[0],d[0]]=reg_mean
    return volcano_map

In [ ]:
# heatmap of centromeres 2hr/DMSO

centromere = [90,65,99,30,111,54,48,126,87,88,30,21,71,23,30,30]

AAave20_cent = get_cent(AAave2_0,volcano=1)

_=plt.imshow((AAave20_cent),cmap = 'bwr',vmax=2,vmin=-2)
_= plt.colorbar(fraction=0.046, pad=0.04)
#plt.title('Centromere 2hr/DMSO')

mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

_=plt.xticks(np.arange(0,16,1),labels = chrom_romlist, rotation =90)
_=plt.yticks(np.arange(0,16,1),labels = chrom_romlist)

# plt.savefig('hic234plots/centromere_map_2hr', dpi=200)

In [ ]:
# heatmap of centromeres 2hr/DMSO

AAave50_cent = get_cent(AAave5_0,volcano=1)

_=plt.imshow((AAave50_cent),cmap = 'bwr',vmax=2,vmin=-2)
_= plt.colorbar(fraction=0.046, pad=0.04)
#plt.title('Centromere 5hr/DMSO')
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

_=plt.xticks(np.arange(0,16,1),labels = chrom_romlist, rotation =90)
_=plt.yticks(np.arange(0,16,1),labels = chrom_romlist)
# plt.savefig('hic234plots/centromere_map_5hr', dpi=200)

# Intrachromosomal Distance Dependence

In [ ]:
from scipy.ndimage import rotate

In [ ]:
# function averaging contact frequencies with respect to distance from a gene within chromosome 

def interchr_length_dependence(matrix,chrome):
    reg = call_region(matrix,chrome,chrome)
    
    m = []
    for i in enumerate(reg):
        m.append(i[1][i[0]+1:])
    df = pd.DataFrame(m)
    nd= df.to_numpy()
    nd = df.fillna(0).values
    nd[nd==0]=np.nan

    m = []
    for i in enumerate(reg):
        a = i[1][0:i[0]]
        m.append(np.flip(a))
    df2 = pd.DataFrame(m)
    nd2 = df2.fillna(0).values
    nd2[nd2==0]=np.nan

    nd_len = []
    nd_len.append(np.nanmean(nd,axis=0))
    nd_len.append(np.nanmean(nd2,axis=0))
    nd_avg = np.average(np.array(nd_len),axis=0)

    return nd_avg

In [ ]:
# Sav_gol filter parameters used to smooth the curves

win_len=5
poly=1

In [ ]:
multiplier = 2
mpl.rc('xtick', labelsize=24*multiplier) 
mpl.rc('ytick', labelsize=24*multiplier)
mpl.rc('figure',figsize = [10.0, 10.0])
# matplotlib.rc('axes',titlesize='large')

font = {'size'   : 24*multiplier}

mpl.rc('font', **font)

plt.rcParams['axes.titley'] = 0.8    # y is in axes-relative coordinates.
plt.rcParams['axes.titlepad'] = -14

plt.rcParams['lines.linewidth'] = 4.0

In [ ]:
# rounding function
def myround(x, base=50):
    return np.int(base * np.round(x/base,1))

In [ ]:
#plots for different chromosomes:

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrIV'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)
plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')
# plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/2)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/2))*5)

#copy
plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
# plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrXV'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
# plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrVII'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
# plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrXII'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrXVI'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrXIII'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrII'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrXIV'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrX'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrXI'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrV'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3.05)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3.05))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrVIII'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3.05)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3.05))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrIX'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrIII'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrVI'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3.01)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3.01))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrI'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2hr/DMSO')
plt.plot(interaction_dist5, color='purple', label='5hr/DMSO')

plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/3))*5)

plt.title(chromo.decode("utf-8")[3:],x=0.7, y=0.85)
 
plt.ylim(0.7,2.5)
plt.yticks((1,2))
plt.vlines(100/5,0.7,2.5,colors='grey',linestyles='--')
# plt.ylabel('Fold Change')
# plt.xlabel('Distance /kB')
# plt.legend()


ax1 = plt.subplot()
ax1.minorticks_on()
ax1.tick_params('both', length=20, width=2, which='major')
ax1.tick_params('both', length=10, width=1, which='minor')
plt.savefig('hic234plots/CFvdistance/'+str(chromo.decode("utf-8")), dpi=200)

## Others

In [ ]:
multiplier = 1
mpl.rc('xtick', labelsize=24*multiplier) 
mpl.rc('ytick', labelsize=24*multiplier)
mpl.rc('figure',figsize = [10.0, 10.0])
# matplotlib.rc('axes',titlesize='large')

font = {'size'   : 24*multiplier}

mpl.rc('font', **font)

plt.rcParams['axes.titley'] = 0.8    # y is in axes-relative coordinates.
plt.rcParams['axes.titlepad'] = -14

plt.rcParams['lines.linewidth'] = 2.5

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrII'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

# plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2 hours')
plt.plot(interaction_dist5, color='purple', label='5 hours')
plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/8)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/8))*5)
plt.ylim(0.75,2.3)
plt.vlines(100/5,0.8,2.2,colors='grey',linestyles='--')
plt.ylabel('Fold Change')
plt.xlabel('Distance /kB')
plt.legend(fontsize="30", loc ="upper right")
plt.savefig('hic234plots/CFvdistance/other/'+str(chromo.decode("utf-8")), dpi=300)

In [ ]:
ratio20_list = [AA22_AA20_nl,AA32_AA30_nl,AA42_AA40_nl]
ratio50_list = [AA25_AA20_nl,AA35_AA30_nl,AA45_AA40_nl]
# b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',
# b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI'
chromo = b'chrIV'
avg_list = []
for i in ratio20_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist2 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist2=interaction_dist2[np.isfinite(interaction_dist2)]
interaction_dist2 = savgol_filter(interaction_dist2,window_length=win_len,polyorder=poly)

avg_list = []
for i in ratio50_list:
    avg_list.append(interchr_length_dependence(i,chromo))
interaction_dist5 = np.nanmean(np.array(avg_list),axis=0)
interaction_dist5=interaction_dist5[np.isfinite(interaction_dist5)]
interaction_dist5 = savgol_filter(interaction_dist5,window_length=win_len,polyorder=poly)

# plt.title(chromo.decode("utf-8"))
plt.plot(interaction_dist2, color='green',label='2 hours')
plt.plot(interaction_dist5, color='purple', label='5 hours')
plt.xticks(np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/5)),np.arange(len(interaction_dist2),step=int(len(interaction_dist2)/5))*5)
plt.ylim(0.775,2.25)

plt.vlines(25/5,0.8,2.2,colors='grey',linestyles='--')
plt.vlines(100/5,0.8,2.2,colors='grey',linestyles='--')
plt.vlines(500/5,0.8,2.2,colors='grey',linestyles='--')

plt.ylabel('Fold Change')
plt.xlabel('Distance /kB')
plt.legend(fontsize="30", loc ="upper right")
plt.savefig('hic234plots/CFvdistance/other/'+str(chromo.decode("utf-8")), dpi=300)